In [ ]:
# Find Matches Demo

In [3]:
% pip install -r requirements.txt

You should consider upgrading via the '/Users/sanhehu/venvs/python/3.8.11/dev_exp_share_venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Standard Library
import csv
import string
import random
from typing import List, Dict

# Third Party Library
import attr
from attrs_mate import AttrsClass
from pathlib_mate import Path

from boto_session_manager import BotoSesManager
from s3pathlib import S3Path, context

import pandas as pd
import awswrangler as wr

from faker import Faker
from rich import print as rprint

## Define Some Utility Functions

First, we need to define some utility functions to keep our code clean.

In [12]:
def add_noise_to_text(text: str, n_noise: int) -> str:
    """
    Randomly add noise character to string.

    Example::

        >>> add_noise_to_text("1234567890", n_noise=3)
        123d56e890
    """
    length = len(text)
    if n_noise > length:
        raise ValueError
    chars = list(text)
    for _ in range(n_noise):
        chars[random.randint(1, length) - 1] = random.choice(string.ascii_lowercase)
    return "".join(chars)


def rand_phone_number() -> str:
    """
    Generate random phone number.

    Example::

        123-456-7890
    """
    numbers = [str(random.randint(0, 9)) for _ in range(10)]
    return "".join(
        numbers[:3]
        + [
            "-",
        ]
        + numbers[3:6]
        + [
            "-",
        ]
        + numbers[6:]
    )


def add_noise_to_phone(phone: str, n_noise: int) -> str:
    """
    Example::

        >>> add_noise_to_phone("111-222-3333", n_noise=1)
        111-222-3353
    """
    chars = list(phone)
    positions = [0, 1, 2, 4, 5, 6, 8, 9, 10]
    for ind in random.sample(positions, n_noise):
        chars[ind] = random.choice(string.digits)
    return "".join(chars)


fake = Faker(locale="en-US")


@attr.define
class Person(AttrsClass):
    """
    A variation of a true person.
    """
    firstname: str = attr.ib()
    lastname: str = attr.ib()
    phone: str = attr.ib()


@attr.define
class TruePerson(AttrsClass):
    """
    represent a concrete human person. he/she could have multiple
    name, phone.
    """
    id: int = attr.ib()
    firstname_list: str = attr.ib()
    lastname_list: str = attr.ib()
    phone_list: str = attr.ib()

    @classmethod
    def random(cls, id: int) -> 'TruePerson':
        firstname = fake.first_name()
        lastname = fake.last_name()
        phone = rand_phone_number()

        firstname_list = [
            firstname,
        ]
        lastname_list = [
            lastname,
        ]
        phone_list = [
            phone,
        ]
        for _ in range(2):
            firstname_list.append(add_noise_to_text(firstname, random.randint(1, 2)))
            lastname_list.append(add_noise_to_text(lastname, random.randint(1, 2)))
            phone_list.append(add_noise_to_phone(phone, random.randint(0, 1)))

        return cls(
            id=id,
            firstname_list=firstname_list,
            lastname_list=lastname_list,
            phone_list=phone_list,
        )

    def to_person(self) -> 'Person':
        return Person(
            firstname=random.choice(self.firstname_list),
            lastname=random.choice(self.lastname_list),
            phone=random.choice(self.phone_list),
        )

In [125]:
print("------ Sample TruePerson ------")
t_person = TruePerson.random(id=1)
rprint(t_person.to_dict())

------ Sample TruePerson ------


{
    'id': 1,
    'firstname_list': ['Christine', 'fhristinp', 'Christone'],
    'lastname_list': ['Molina', 'Mopina', 'Mxliya'],
    'phone_list': ['578-774-5954', '578-774-5954', '578-774-5954']
}

In [126]:
print("------ Sample Person ------")
person = t_person.to_person()
rprint(person.to_dict())

------ Sample Person ------


{'firstname': 'Christine', 'lastname': 'Mopina', 'phone': '578-774-5954'}

## Define Your Test Environment Configuration

We need to define the AWS Credential, the AWS S3 bucket we use to store the data and the Glue Catalog database / table name.

In [118]:
# Define AWS boto3 credentials for session
bsm = BotoSesManager(profile_name="aws_data_lab_sanhe_us_east_2")
context.attach_boto_session(boto_ses=bsm.boto_ses)

# Where you want to store your test data locally
dir_here = Path.cwd()
path_all_csv = Path(dir_here, "all.csv")
path_records_csv = Path(dir_here, "records.csv")
path_labels_csv = Path(dir_here, "labels.csv")
path_tests_csv = Path(dir_here, "tests.csv")
path_predicts_csv = Path(dir_here, "predicts.csv")
path_compares_csv = Path(dir_here, "compares.csv")

# Where you want to store your test data on S3
s3path_prefix = S3Path.from_s3_uri(
    "s3://aws-data-lab-sanhe-for-everything-us-east-2/poc/2022-05-18-glue-find-matches/find-matches/"
)

s3path_records = S3Path(s3path_prefix, "records")
s3path_labels = S3Path(s3path_prefix, "labels")
s3path_tests = S3Path(s3path_prefix, "tests")
s3path_predicts = S3Path(s3path_prefix, "predicts")

# Glue Catalog config
db_name = "learn_glue_find_matches"
tb_name_records = "records"
tb_name_labels = "labels"
tb_name_tests = "tests"
tb_name_predicts = "predicts"


## Define Your Test Dataset Configuration

We need to define some statistics information about our test dataset

- ``n_label_set_id``: based on [official document](https://docs.aws.amazon.com/glue/latest/dg/machine-learning.html), the training dataset should be split into "chunks". Each chunk is a "labeling_set". Within each "labeling_set", you could have many records (no more than 300, it helps the ML training fast and efficient), and those records can be clustered into different "group". Each group should have a "label". Records belongs to the same "group" considered as a "match". With in each "labeling_set", you should have some "match group" that has many records, and also have some "non match group" that has only one record, which indicate that this record doesn't match any of the others. **This config defines the total number of labeling set you want to generate**.
- ``n_label_per_set``: The number of records in each labeling set.
- ``n_sample_list``: records in each labeling set will be divided into different group, this config defines the number of records for each group.


In [101]:
n_label_set_id = 1000
n_label_per_set = 20
n_sample_list = [10, 6, 3, 1]

if 1 not in n_sample_list:
    raise ValueError(
        "You have to has at least ONE group has only ONE member!"
        "So the find matches ML can learn from it."
    )

if sum(n_sample_list) != n_label_per_set:
    raise ValueError


## Generate Test Data

We want to write some code to generate the dataset and:

1. store them locally for visual inspect
2. store them to S3 for Glue Find Matches ML job

In [127]:
columns_all = "labeling_set_id,label,id,tid,firstname,lastname,phone".split(",")

def s1_generate_dataset() -> pd.DataFrame:
    # global unique identifier for each records
    id = 0

    # the True Person id, if multiple records has the same tid,
    # they should considered as a match
    # We can use this value to validate the ML predict
    tid = 0

    rows = list()
    for label_set_id in range(1, 1 + n_label_set_id):
        for label, n_sample in enumerate(n_sample_list, start=1):
            tid += 1
            true_person = TruePerson.random(tid)
            for _ in range(n_sample):
                id += 1
                person = true_person.to_person()
                row = dict(
                    labeling_set_id=f"LabelSetId-{label_set_id}",
                    label=f"Label-{label}",
                    id=f"PersonId-{str(id).zfill(5)}",
                    tid=f"TrueId-{tid}",
                    firstname=person.firstname,
                    lastname=person.lastname,
                    phone=person.phone,
                )
                rows.append(row)

    df = pd.DataFrame(
        rows,
        columns=columns_all,
    )
    return df

df_all = s1_generate_dataset()

Now, let's take a look at the dataset.

In [128]:
print("------ all data ------")
print(f"{df_all.shape[0]} rows, {df_all.shape[1]} columns")
df_all.head(5)


------ all data ------
20000 rows, 7 columns


,labeling_set_id,label,id,tid,firstname,lastname,phone
0,LabelSetId-1,Label-1,PersonId-00001,TrueId-1,John,aadden,672-615-3608
1,LabelSetId-1,Label-1,PersonId-00002,TrueId-1,Jjcn,aadden,642-615-3608
2,LabelSetId-1,Label-1,PersonId-00003,TrueId-1,Joln,aadden,602-615-3608
3,LabelSetId-1,Label-1,PersonId-00004,TrueId-1,Jjcn,Mbddvn,672-615-3608
4,LabelSetId-1,Label-1,PersonId-00005,TrueId-1,Jjcn,aadden,672-615-3608


In [129]:
df_train = df_all.sample(frac=0.7)
df_train = df_train.sort_values(by="id")
print("------ train data ------")
print(f"{df_train.shape[0]} rows, {df_train.shape[1]} columns")
df_train.head(5)


------ train data ------
14000 rows, 7 columns


,labeling_set_id,label,id,tid,firstname,lastname,phone
0,LabelSetId-1,Label-1,PersonId-00001,TrueId-1,John,aadden,672-615-3608
1,LabelSetId-1,Label-1,PersonId-00002,TrueId-1,Jjcn,aadden,642-615-3608
2,LabelSetId-1,Label-1,PersonId-00003,TrueId-1,Joln,aadden,602-615-3608
4,LabelSetId-1,Label-1,PersonId-00005,TrueId-1,Jjcn,aadden,672-615-3608
7,LabelSetId-1,Label-1,PersonId-00008,TrueId-1,John,Mbddvn,672-615-3608


In [130]:
columns_records = "id,firstname,lastname,phone".split(",")
columns_labels = "labeling_set_id,label,id,firstname,lastname,phone".split(",")

df_records = df_train.loc[:, columns_records]
df_labels = df_train.loc[:, columns_labels]
df_tests = df_all.loc[:, columns_records]

In [131]:
print("------ the 'records' csv file ------")
print(f"{df_records.shape[0]} rows, {df_records.shape[1]} columns")
df_records.head(10)

------ the 'records' csv file ------
14000 rows, 4 columns


,id,firstname,lastname,phone
0,PersonId-00001,John,aadden,672-615-3608
1,PersonId-00002,Jjcn,aadden,642-615-3608
2,PersonId-00003,Joln,aadden,602-615-3608
4,PersonId-00005,Jjcn,aadden,672-615-3608
7,PersonId-00008,John,Mbddvn,672-615-3608
8,PersonId-00009,John,Mbddvn,672-615-3608
9,PersonId-00010,Jjcn,Mbddvn,602-615-3608
10,PersonId-00011,Joel,Morshall,546-359-8339
11,PersonId-00012,Joel,Marshall,546-359-8339
12,PersonId-00013,Jyxl,Morshall,546-359-8339


In [132]:
print("------ the 'labels' csv file ------")
print(f"{df_labels.shape[0]} rows, {df_labels.shape[1]} columns")
df_labels.head(10)

------ the 'labels' csv file ------
14000 rows, 6 columns


,labeling_set_id,label,id,firstname,lastname,phone
0,LabelSetId-1,Label-1,PersonId-00001,John,aadden,672-615-3608
1,LabelSetId-1,Label-1,PersonId-00002,Jjcn,aadden,642-615-3608
2,LabelSetId-1,Label-1,PersonId-00003,Joln,aadden,602-615-3608
4,LabelSetId-1,Label-1,PersonId-00005,Jjcn,aadden,672-615-3608
7,LabelSetId-1,Label-1,PersonId-00008,John,Mbddvn,672-615-3608
8,LabelSetId-1,Label-1,PersonId-00009,John,Mbddvn,672-615-3608
9,LabelSetId-1,Label-1,PersonId-00010,Jjcn,Mbddvn,602-615-3608
10,LabelSetId-1,Label-2,PersonId-00011,Joel,Morshall,546-359-8339
11,LabelSetId-1,Label-2,PersonId-00012,Joel,Marshall,546-359-8339
12,LabelSetId-1,Label-2,PersonId-00013,Jyxl,Morshall,546-359-8339


In [133]:
print("------ test data ------")
print(f"{df_tests.shape[0]} rows, {df_tests.shape[1]} columns")
df_tests.head(10)

------ test data ------
20000 rows, 4 columns


,id,firstname,lastname,phone
0,PersonId-00001,John,aadden,672-615-3608
1,PersonId-00002,Jjcn,aadden,642-615-3608
2,PersonId-00003,Joln,aadden,602-615-3608
3,PersonId-00004,Jjcn,Mbddvn,672-615-3608
4,PersonId-00005,Jjcn,aadden,672-615-3608
5,PersonId-00006,Jjcn,aadden,642-615-3608
6,PersonId-00007,Jjcn,Madden,602-615-3608
7,PersonId-00008,John,Mbddvn,672-615-3608
8,PersonId-00009,John,Mbddvn,672-615-3608
9,PersonId-00010,Jjcn,Mbddvn,602-615-3608


In [137]:
# Write to Local
quoting = csv.QUOTE_NONNUMERIC
to_csv_kwargs = dict(sep=",", index=False, header=True, quoting=quoting)

df_all.to_csv(path_all_csv, **to_csv_kwargs)
df_records.to_csv(path_records_csv, **to_csv_kwargs)
df_labels.to_csv(path_labels_csv, **to_csv_kwargs)
df_tests.to_csv(path_tests_csv, **to_csv_kwargs)

# Create catalog table if not exists
databases = wr.catalog.databases(boto3_session=bsm.boto_ses)
if db_name not in databases["Database"].to_list():
    print(f"create database {db_name}")
    wr.catalog.create_database(db_name)

tables = wr.catalog.tables(database=db_name, boto3_session=bsm.boto_ses)

# Write to S3
s3path_prefix.delete_if_exists()

to_csv_kwargs = dict(
    dataset=True,
    sep=",",
    index=False,
    header=True,
    boto3_session=bsm.boto_ses,
)

wr.s3.to_csv(
    df=df_records,
    path=s3path_records.uri,
    **to_csv_kwargs
)
s3path = s3path_records.iter_objects().one()
print(f"created {s3path.uri}")
print(f"  preview at {s3path.console_url}")

wr.s3.to_csv(
    df=df_labels,
    path=s3path_labels.uri,
    **to_csv_kwargs
)
s3path = s3path_labels.iter_objects().one()
print(f"created {s3path.uri}")
print(f"  preview at {s3path.console_url}")

wr.s3.to_csv(
    df=df_tests,
    path=s3path_tests.uri,
    **to_csv_kwargs
)
s3path = s3path_tests.iter_objects().one()
print(f"created {s3path.uri}")
print(f"  preview at {s3path.console_url}")

created s3://aws-data-lab-sanhe-for-everything-us-east-2/poc/2022-05-18-glue-find-matches/find-matches/records/f0ca73488f37447baff5ada06cc71f1e.csv
  preview at https://console.aws.amazon.com/s3/object/aws-data-lab-sanhe-for-everything-us-east-2?prefix=poc/2022-05-18-glue-find-matches/find-matches/records/f0ca73488f37447baff5ada06cc71f1e.csv
created s3://aws-data-lab-sanhe-for-everything-us-east-2/poc/2022-05-18-glue-find-matches/find-matches/labels/b52b36fd1ba647c799d57049ed8dd275.csv
  preview at https://console.aws.amazon.com/s3/object/aws-data-lab-sanhe-for-everything-us-east-2?prefix=poc/2022-05-18-glue-find-matches/find-matches/labels/b52b36fd1ba647c799d57049ed8dd275.csv
created s3://aws-data-lab-sanhe-for-everything-us-east-2/poc/2022-05-18-glue-find-matches/find-matches/tests/b5577eb9e7cd43118d7b8d70765853e6.csv
  preview at https://console.aws.amazon.com/s3/object/aws-data-lab-sanhe-for-everything-us-east-2?prefix=poc/2022-05-18-glue-find-matches/find-matches/tests/b5577eb9e7c

## Train the ML Model

1. create the Glue Database, and Glue Table (better to create it manually in console to avoid weird error)

![](./glue-table.png)

2. create the Glue ML job, follow this guide: https://docs.aws.amazon.com/glue/latest/dg/machine-learning-transform-tutorial.html

3. after you provide the label, click the "Estimate the Quality button". Then your ML model is ready to use. Take a note of the Glue ML job transformation ID from the console.

4. Create a Glue Job with Jupyter Notebook in the console, you can find an example at ``glue-etl-job.ipynb``. It reads the data from S3, run the transformation job, and store the results in S3

## Validate The Prediction

Now we want to download the predict results and compare to the Ground Truth TruePerson ID.

In [138]:
def download_the_predicts():
    """
    Merge and download the predict output data from S3 to Local.
    The Glue ML job usually use multiple worker and dump the results
    to many small files in parallel,
    """
    df_list = list()
    for s3path in S3Path(s3path_prefix, "predict").iter_objects():
        with s3path.open("r") as f:
            df = pd.read_csv(f)
            df_list.append(df)
    df_predict = pd.concat(df_list, axis=0)
    # re order by ID to ensure the order
    df_predict = df_predict.sort_values(by="id")
    df_predict.to_csv(
        path_predicts_csv,
        sep=",",
        index=False,
    )

download_the_predicts()

In [139]:
df_predict = pd.read_csv(path_predicts_csv)

In [140]:
print("------ Predict Results ------")
print(f"{df_predict.shape[0]} rows, {df_predict.shape[1]} columns")
df_predict.head(10)

------ Predict Results ------
20000 rows, 6 columns


,id,firstname,lastname,phone,match_id,match_confidence_score
0,PersonId-00001,John,aadden,672-615-3608,0,1.0
1,PersonId-00002,Jjcn,aadden,642-615-3608,0,1.0
2,PersonId-00003,Joln,aadden,602-615-3608,0,1.0
3,PersonId-00004,Jjcn,Mbddvn,672-615-3608,0,1.0
4,PersonId-00005,Jjcn,aadden,672-615-3608,0,1.0
5,PersonId-00006,Jjcn,aadden,642-615-3608,0,1.0
6,PersonId-00007,Jjcn,Madden,602-615-3608,0,1.0
7,PersonId-00008,John,Mbddvn,672-615-3608,0,1.0
8,PersonId-00009,John,Mbddvn,672-615-3608,0,1.0
9,PersonId-00010,Jjcn,Mbddvn,602-615-3608,0,1.0


In [141]:
# merge the predicted match id and the true person id
# so we can compare visually
df_compare = df_predict.copy()
df_compare["tid"] = df_all["tid"]
df_compare = df_compare["id,firstname,lastname,phone,tid,match_id,match_confidence_score".split(",")]
df_compare.to_csv(
    path_compares_csv,
    sep=",",
    index=False,
)

In [143]:
print("You can visually check the 'tid' and 'match_id' columns")
df_compare.head(50)

You can visually check the 'tid' and 'match_id' columns


,id,firstname,lastname,phone,tid,match_id,match_confidence_score
0,PersonId-00001,John,aadden,672-615-3608,TrueId-1,0,1.0
1,PersonId-00002,Jjcn,aadden,642-615-3608,TrueId-1,0,1.0
2,PersonId-00003,Joln,aadden,602-615-3608,TrueId-1,0,1.0
3,PersonId-00004,Jjcn,Mbddvn,672-615-3608,TrueId-1,0,1.0
4,PersonId-00005,Jjcn,aadden,672-615-3608,TrueId-1,0,1.0
5,PersonId-00006,Jjcn,aadden,642-615-3608,TrueId-1,0,1.0
6,PersonId-00007,Jjcn,Madden,602-615-3608,TrueId-1,0,1.0
7,PersonId-00008,John,Mbddvn,672-615-3608,TrueId-1,0,1.0
8,PersonId-00009,John,Mbddvn,672-615-3608,TrueId-1,0,1.0
9,PersonId-00010,Jjcn,Mbddvn,602-615-3608,TrueId-1,0,1.0
